In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('../raw_data/GgSheetData.csv')

Kiểm tra trùng lặp 'content'

In [22]:
# Kiểm tra số lượng tin nhắn trùng lặp theo cột "Content"
num_total = df.shape[0] 
num_content_unique = df['content'].nunique()
num_content_duplicates = num_total - num_content_unique

print(f"Tổng số tin nhắn: {num_total}")
print(f"Số lượng nội dung tin nhắn duy nhất: {num_content_unique}")
print(f"Số lượng nội dung tin nhắn bị trùng lặp: {num_content_duplicates}")

# Các nội dung bị trùng lặp (nội dung xuất hiện nhiều hơn 1 lần)
duplicate_contents = df['content'].value_counts()
duplicate_contents = duplicate_contents[duplicate_contents > 1]
if not duplicate_contents.empty:
    print("\nMột số nội dung tin nhắn bị trùng lặp:")
    print(duplicate_contents)
else:
    print("Không có nội dung tin nhắn nào bị trùng lặp.")


Tổng số tin nhắn: 251
Số lượng nội dung tin nhắn duy nhất: 250
Số lượng nội dung tin nhắn bị trùng lặp: 1

Một số nội dung tin nhắn bị trùng lặp:
content
TB: Tu 10/6/17, QK vui long su dung TK DDA 104005886693 gan voi the ATM E-Partner thay cho TK 711A50424587 voi so du duoc giu nguyen. Cam on QK. LH 1900558868    2
Name: count, dtype: int64


Loại bỏ trùng lặp

In [23]:
# Loại bỏ tin nhắn trùng lặp, chỉ giữ lại 1 lần duy nhất cho mỗi nội dung "Content"
df = df.drop_duplicates(subset=['content'], keep='first').reset_index(drop=True)

# Kiểm tra lần 2
num_total = df.shape[0]
num_content_unique = df['content'].nunique()
num_content_duplicates = num_total - num_content_unique

print(f"Tổng số tin nhắn: {num_total}")
print(f"Số lượng nội dung tin nhắn duy nhất: {num_content_unique}")
print(f"Số lượng nội dung tin nhắn bị trùng lặp: {num_content_duplicates}")

Tổng số tin nhắn: 250
Số lượng nội dung tin nhắn duy nhất: 250
Số lượng nội dung tin nhắn bị trùng lặp: 0


## Xử lý các từ khoá đại diện
- [NAME], [CCCD], [LOCATION], [PHONE_NUMBER], [LINK], [MONEY_AMOUNT], [BANK_ACCOUNT], [DATETIME], [PRIVATE_INFO]
- Dữ liệu từ Google Sheet là dữ liệu được thu thập thủ công từ các nguồn như ảnh/văn bản về tin nhắn độc hại được đăng lên mạng, cho nên sẽ có một số thông tin bị làm mờ/che nhằm bảo mật thông tin cá nhân. 
- Trong quá trình thu thập, nhóm chúng em đã quy định các từ khoá đại diện như trên nhằm thay thế cho các thông tin không có được. 
- Nhóm hiện đang tìm giải pháp cho vấn đề trên

### Thống kê số lượng mẫu dữ liệu rơi vào trường hợp 'từ khoá đại diện'

In [24]:
# Danh sách các từ khoá đại diện
placeholder_keywords = [
    '[NAME]', '[CCCD]', '[LOCATION]', '[PHONE_NUMBER]', 
    '[LINK]', '[MONEY_AMOUNT]', '[BANK_ACCOUNT]', '[DATETIME]', '[PRIVATE_INFO]'
]

# Tìm các dòng có chứa bất kỳ từ khoá đại diện nào trong content
import re
pattern = '|'.join(map(re.escape, placeholder_keywords))
mask_has_placeholder = df['content'].str.contains(pattern, regex=True, na=False)

# Lấy các mẫu dữ liệu chứa từ khoá đại diện
df_placeholder = df[mask_has_placeholder]

print(f"Số lượng mẫu dữ liệu chứa từ khoá đại diện: {df_placeholder.shape[0]}")
print("Một số ví dụ (tối đa 5 dòng):")
print(df_placeholder[['content']].head(5))


Số lượng mẫu dữ liệu chứa từ khoá đại diện: 30
Một số ví dụ (tối đa 5 dòng):
                                             content
0  [TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông...
2  Western Union TB: Vietcombank: [BANK_ACCOUNT]....
6  [NAME] đang làm gì vậy? Dạo này công việc thế ...
7  [NAME] ơi anh [NAME] nè, hôm nay vô tình gặp c...
8     [NAME] dạo này khoẻ không, công việc ổn chứ ? 


### Hướng giải quyết: Data Augmentation
- Thay thế các placeholder nói trên bằng dữ liệu thực/giả thực.
- Vì số lượng các tin nhắn như trên không nhiều (30 mẫu), cho nên chúng em tiến hành thay thế từng câu một.

In [25]:
import re
# Dictionary chứa mapping từ nội dung gốc -> nội dung đã thay thế
# Key: nội dung gốc chứa placeholder
# Value: nội dung đã được thay thế bằng dữ liệu thực/giả thực

manual_replacements = {
    # === TIN NHẮN ĐE DỌA THU NỢ / LỪA ĐẢO TÀI CHÍNH ===
    
    # 1. Tin nhắn giả mạo trung tâm phòng chống gian lận
    '[TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông/Bà [NAME] Trước 17h ngày hôm nay không thanh toán dứt điểm số tiền quá hạn 910.046 VNĐ. Chúng tôi đơn phương chấm dứt hợp đồng vay và gửi hồ sơ về Nhân viên thu hồi nợ tại nhà. Cùng Chính Quyền Địa Phương Huyện [LOCATION] xuống nhà tại địa chỉ cư trú [LOCATION]. Thanh lý hết tổng số tiền 19.137.909 VNĐ. Mọi chi phí phát sinh Ông/bà hoàn toàn chịu trách nhiệm.':
    '[TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông/Bà Nguyễn Văn Minh Trước 17h ngày hôm nay không thanh toán dứt điểm số tiền quá hạn 910.046 VNĐ. Chúng tôi đơn phương chấm dứt hợp đồng vay và gửi hồ sơ về Nhân viên thu hồi nợ tại nhà. Cùng Chính Quyền Địa Phương Huyện Bình Chánh xuống nhà tại địa chỉ cư trú Quận Gò Vấp, TP.HCM. Thanh lý hết tổng số tiền 19.137.909 VNĐ. Mọi chi phí phát sinh Ông/bà hoàn toàn chịu trách nhiệm.',
    
    # 2. Tin nhắn lời nhắn thoại Viettel với [BANK]
    '[TB] Tin ich Loi nham thoai cua Viettel: Quy khach co loi nhan tu TB 0848836182 vao luc 08:09 27/03/2025. Voi noi dung: kính chào quý khách đây là cuộc gọi tự động từ ngân hàng [BANK] nhằm hỗ trợ thông tin khoản vay quý khách vui lòng nghe hết cuộc gọi và làm theo hướng dẫn để hệ thống ghi nhận cam kết thanh toán quý khách hiện có khoản vay quán hạn cao ngày nhất là 450 ngày tổng số tiền quá hạn cần thanh toán trong hôm nay là 6367474 đồng quý khách vui lòng thanh toán ngay hôm nay để tránh bị chuyển nhóm nợ cao hơn và ảnh hưởng đến việc vay vốn tại các tổ chức tín dụng nếu quý khách đã thanh toán vui lòng bấm phím một cam kết thanh toán ngay hôm nay vui lòng nhấn phím hai cần gặp tổng đài viên vui lòng nhấn phím ba kính chúc quý khách một ngày tốt lành xin cảm ơn và tạm biệt quý khách kính chào quý khách đây là cuộc gọi tự động từ ngân hàng... De nghe loi nhan thoai, vui long bam goi 106226 (mien phi). ':
    '[TB] Tin ich Loi nham thoai cua Viettel: Quy khach co loi nhan tu TB 0848836182 vao luc 08:09 27/03/2025. Voi noi dung: kính chào quý khách đây là cuộc gọi tự động từ ngân hàng FE Credit nhằm hỗ trợ thông tin khoản vay quý khách vui lòng nghe hết cuộc gọi và làm theo hướng dẫn để hệ thống ghi nhận cam kết thanh toán quý khách hiện có khoản vay quán hạn cao ngày nhất là 450 ngày tổng số tiền quá hạn cần thanh toán trong hôm nay là 6367474 đồng quý khách vui lòng thanh toán ngay hôm nay để tránh bị chuyển nhóm nợ cao hơn và ảnh hưởng đến việc vay vốn tại các tổ chức tín dụng nếu quý khách đã thanh toán vui lòng bấm phím một cam kết thanh toán ngay hôm nay vui lòng nhấn phím hai cần gặp tổng đài viên vui lòng nhấn phím ba kính chúc quý khách một ngày tốt lành xin cảm ơn và tạm biệt quý khách kính chào quý khách đây là cuộc gọi tự động từ ngân hàng... De nghe loi nhan thoai, vui long bam goi 106226 (mien phi). ',
    
    # 3. Tin nhắn Western Union lừa đảo
    'Western Union TB: Vietcombank: [BANK_ACCOUNT]. [NAME]. Ref +19.56 USD. Nhận 500.000 VND. Ngay 02/02/2025. Mgd: 1057425286. Nd: COC TIEN HANG. Quý khách nhận tiền VND vào website: https://sites.google.com/vỉew/chuyennhantiennhanhquocte24h7':
    'Western Union TB: Vietcombank: 0071000986547. Trần Thị Lan. Ref +19.56 USD. Nhận 500.000 VND. Ngay 02/02/2025. Mgd: 1057425286. Nd: COC TIEN HANG. Quý khách nhận tiền VND vào website: https://sites.google.com/vỉew/chuyennhantiennhanhquocte24h7',
    
    # 4. Cảnh báo nợ với đầy đủ thông tin cá nhân
    'CANH BAO LAN CUOI!!! Trong 24H nua Ong/Ba [NAME] CMND: [CCCD] SDT: [PHONE_NUMBER] phai lien he gap SDT/ZALO: [PHONE_NUMBER] gap [NAME] de THOA THUAN - GIAM NO. Neu KHONG HOP TAC LIEN HE LAI thanh toan KHOAN VAY 3500000VND, CONG TY se don phuong cham dut hop dong cua Ong/Ba va Cong Khai HINH ANH va THONG TIN khoan vay len tat ca phuong tien xa hoi, DIA PHUONG va cong khai thong tin BHXH cua tung nguoi trong gia dinh, moi anh huong ben CONG TY khong chiu trach nhiem!!!':
    'CANH BAO LAN CUOI!!! Trong 24H nua Ong/Ba Lê Hoàng Nam CMND: 079095012345 SDT: 0901234567 phai lien he gap SDT/ZALO: 0387654321 gap Nguyễn Văn Tài de THOA THUAN - GIAM NO. Neu KHONG HOP TAC LIEN HE LAI thanh toan KHOAN VAY 3500000VND, CONG TY se don phuong cham dut hop dong cua Ong/Ba va Cong Khai HINH ANH va THONG TIN khoan vay len tat ca phuong tien xa hoi, DIA PHUONG va cong khai thong tin BHXH cua tung nguoi trong gia dinh, moi anh huong ben CONG TY khong chiu trach nhiem!!!',
    
    # === TIN NHẮN GIẢ MẠO TỪ NGƯỜI QUEN ===
    
    # 5. Tin nhắn hỏi thăm giả mạo người quen (làm quen lạ)
    '[NAME] đang làm gì vậy? Dạo này công việc thế nào rồi? ':
    'Hương đang làm gì vậy? Dạo này công việc thế nào rồi? ',
    
    # 6. Tin nhắn xin số điện thoại
    '[NAME] ơi anh [NAME] nè, hôm nay vô tình gặp chị [NAME] nên xin được số em':
    'Linh ơi anh Tuấn nè, hôm nay vô tình gặp chị Mai nên xin được số em',
    
    # 7. Tin nhắn hỏi thăm sức khỏe
    '[NAME] dạo này khoẻ không, công việc ổn chứ ? ':
    'Thảo dạo này khoẻ không, công việc ổn chứ ? ',
    
    # 8. Tin nhắn chào hỏi gia đình
    'Chào [NAME]! Bạn và gia đình dạo này khoẻ chứ?':
    'Chào Phương! Bạn và gia đình dạo này khoẻ chứ?',
    
    # 9. Tin nhắn lâu không gặp
    'Chào [NAME]. Lâu rồi không gặp. dạo này công việc vẫn ổn chứ':
    'Chào Hùng. Lâu rồi không gặp. dạo này công việc vẫn ổn chứ',
    
    # === TIN NHẮN ĐÒI NỢ / NHẮC NỢ ===
    
    # 10. Tin nhắn đòi nợ với SĐT
    'Nhờ nhắn lại [NAME] 24/05/2004. SĐT: [PHONE_NUMBER], ra thanh toán khoản nợ 2.040.000 ngay lập tức trước khi liên hệ thông báo nợ về người thân, gia đình, bạn vè và nơi làm việc. Liên hệ Zalo: 0708922540 để được giải quyết giảm lãi thanh toán nợ':
    'Nhờ nhắn lại Trần Văn Đức 24/05/2004. SĐT: 0912345678, ra thanh toán khoản nợ 2.040.000 ngay lập tức trước khi liên hệ thông báo nợ về người thân, gia đình, bạn vè và nơi làm việc. Liên hệ Zalo: 0708922540 để được giải quyết giảm lãi thanh toán nợ',
    
    # 11. Tin nhắn giả vờ có việc gấp
    'Em [NAME] day. Goi lai vao so [PHONE_NUMBER] em co viec gap.':
    'Em Hà day. Goi lai vao so 0978123456 em co viec gap.',
    
    # === TIN NHẮN LỪA ĐẢO NGÂN HÀNG / QUAY SỐ TRÚNG THƯỞNG ===
    
    # 12. VPBank thông báo quay số trúng thưởng (mẫu 1)
    'Ngan Hang VPBANK THONG BAO. Chuc Mung Tai Khoan Cua [NAME] Nhan Duoc 1 Luot "QUAY SO - MIEN PHI", Nhan Ngay 1 Trong 3 Phan Qua Gom: 1 iPhone 11 512GB. 1 Tivi LG 4k 55inch. 10 Trieu Dong Vao Tai Khoan. Truy Cap Website: https://login-onlien.com.vn/ De Thuc Hien Luot QUAY SO MAY MAN MIEN PHI.Goi Vao 1900 54 54 15.Xin Cam On!':
    'Ngan Hang VPBANK THONG BAO. Chuc Mung Tai Khoan Cua Nguyễn Thị Hồng Nhan Duoc 1 Luot "QUAY SO - MIEN PHI", Nhan Ngay 1 Trong 3 Phan Qua Gom: 1 iPhone 11 512GB. 1 Tivi LG 4k 55inch. 10 Trieu Dong Vao Tai Khoan. Truy Cap Website: https://login-onlien.com.vn/ De Thuc Hien Luot QUAY SO MAY MAN MIEN PHI.Goi Vao 1900 54 54 15.Xin Cam On!',
    
    # 13. VPBank thông báo quay số trúng thưởng (mẫu 2)
    'Ngan Hang VPBANK THONG BAO. Chuc Mung Tai Khoan Cua [NAME] Nhan Duoc 1 Luot "QUAY SO - MIEN PHI", Nhan Ngay 1 Trong 3 Phan Qua Gom: 1 iPhone 11 512GB. 1 Tivi LG 4K 55inch. 10 Trieu Dong Vao Tai Khoan. Truy Cap Website: https://login-online.com.vn/ De Thuc Hien Luot QUAY SO MAY MAN MIEN PHI.Goi Vao 1900 54 54 15.Xin Cam On!':
    'Ngan Hang VPBANK THONG BAO. Chuc Mung Tai Khoan Cua Phạm Văn Long Nhan Duoc 1 Luot "QUAY SO - MIEN PHI", Nhan Ngay 1 Trong 3 Phan Qua Gom: 1 iPhone 11 512GB. 1 Tivi LG 4K 55inch. 10 Trieu Dong Vao Tai Khoan. Truy Cap Website: https://login-online.com.vn/ De Thuc Hien Luot QUAY SO MAY MAN MIEN PHI.Goi Vao 1900 54 54 15.Xin Cam On!',
    
    # === TIN NHẮN GIẢ MẠO CƠ QUAN NHÀ NƯỚC ===
    
    # 14. Giả mạo công chức thuế
    'Chào chị. Tôi là [NAME] - công chức tại Cơ quan Thuế vừa trao đổi điện thoại với chị. Để làm tờ khai thuế online, mời chị truy cập và tải app của Cơ quan tại link: thuedientuu.vn Sau khi đã tải app xong, hãy phản hồi lại tin nhắn này để chúng tôi trao đổi và hướng dẫn các bước tiếp theo.':
    'Chào chị. Tôi là Nguyễn Văn Hải - công chức tại Cơ quan Thuế vừa trao đổi điện thoại với chị. Để làm tờ khai thuế online, mời chị truy cập và tải app của Cơ quan tại link: thuedientuu.vn Sau khi đã tải app xong, hãy phản hồi lại tin nhắn này để chúng tôi trao đổi và hướng dẫn các bước tiếp theo.',
    
    # === TIN NHẮN GIẢ MẠO ĐIỆN LỰC ===
    
    # 15. Điện lực thông báo hóa đơn (mẫu 1)
    'DIEN LUC thong bao khach hang [NAME] [PERSONAL_INFO], co tong so hoa don 1, voi tong tien: 160956dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay. ':
    'DIEN LUC thong bao khach hang Trần Văn Bình PE15HN0234567, co tong so hoa don 1, voi tong tien: 160956dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay. ',
    
    # 16. Điện lực thông báo hóa đơn (mẫu 2)
    'DIEN LUC thong bao khach hang [NAME] [PERSONAL_INFO], co tong so hoa don 1, voi tong tien: 607262dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay. ':
    'DIEN LUC thong bao khach hang Lê Thị Hoa PE07QN0345678, co tong so hoa don 1, voi tong tien: 607262dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay. ',
    
    # 17. Điện lực thông báo hóa đơn (mẫu 3) - với [PRIVATE_INFO]
    'DIEN LUC thong bao khach hàng [NAME][PRIVATE_INFO], co tong so hoa don 1, voi tong tien 607262dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay.':
    'DIEN LUC thong bao khach hàng Nguyễn Hoàng AnhPE12DN0456789, co tong so hoa don 1, voi tong tien 607262dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay.',
    
    # 18. Điện lực thông báo hóa đơn (mẫu 4)
    'DIEN LUC thong bao khach hang [NAME] PC06NN0123302.NCC02, co tong so hoa don 1, voi tong tien: 607262dong. De nghi Quy khach thanh toan trong ong 2 ngay ke tu khi nhan luoc tin nhan. Neu Quy khach la thanh toan, vui long bo qua tin nhan nay.':
    'DIEN LUC thong bao khach hang Võ Văn Thành PC06NN0123302.NCC02, co tong so hoa don 1, voi tong tien: 607262dong. De nghi Quy khach thanh toan trong ong 2 ngay ke tu khi nhan luoc tin nhan. Neu Quy khach la thanh toan, vui long bo qua tin nhan nay.',
    
    # 19. Điện lực thông báo hóa đơn (mẫu 5)
    'DIEN LUC thong bao khach hang [NAME], [PRIVATE_INFO], co tong so hoa don 1, voi tong tien: 160956dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay.':
    'DIEN LUC thong bao khach hang Đỗ Thị Thanh, PE08BD0567890, co tong so hoa don 1, voi tong tien: 160956dong. De nghi Quy khach thanh toan trong vong 2 ngay ke tu khi nhan duoc tin nhan. Neu Quy khach da thanh toan, vui long bo qua tin nhan nay.',
    
    # === TIN NHẮN LỪA ĐẢO TÌNH DỤC / HẸN HÒ ===
    
    # 20. Tin nhắn quảng cáo dịch vụ người lớn với [LINK]
    'Bạn đang muốn tìm tình một đêm? Để có những cảm xúc thăng hoa, thoả mãn nhu cầu sinh lý, trải nghiệm cảm giác khoái lạc hãy liên hệ tôi: [LINK]':
    'Bạn đang muốn tìm tình một đêm? Để có những cảm xúc thăng hoa, thoả mãn nhu cầu sinh lý, trải nghiệm cảm giác khoái lạc hãy liên hệ tôi: https://t.me/henho24h',
    
    # 21. Tin nhắn mại dâm với [PHONE_NUMBER]
    'Em la nu sinh vien 21 tuoi MU0N KIEM chut Tjen trang TRAI CU0C S0NG EM C0 the den lam tinh khu vuc nao cung DU0C GIA CA Uu DAI Add ZAL0 e: [PHONE_NUMBER] QkR':
    'Em la nu sinh vien 21 tuoi MU0N KIEM chut Tjen trang TRAI CU0C S0NG EM C0 the den lam tinh khu vuc nao cung DU0C GIA CA Uu DAI Add ZAL0 e: 0367891234 QkR',
    
    # 22. Tin nhắn mại dâm với [LINK] Telegram
    'Anh co can GAI LAM tjnh K0? E di lam them kiem tien tieu vat EM 20 T DANG NG0N, B0DY chuan Den tan noi LAM TJNH Ket Ban Telegram: [LINK] QNfT':
    'Anh co can GAI LAM tjnh K0? E di lam them kiem tien tieu vat EM 20 T DANG NG0N, B0DY chuan Den tan noi LAM TJNH Ket Ban Telegram: https://t.me/gaixinh_hcm QNfT',
    
    # === TIN NHẮN GIẢ MẠO NHÀ MẠNG ===
    
    # 23. Tin nhắn lời nhắn từ số điện thoại
    'Bạn nhận được lời nhắn từ [PHONE_NUMBER] vào lúc 13:08 30/09/2023 với nội dung: xin thông báo số thuê bao của quý khách sẽ bị khoá trong 2 giờ tới vui lòng đến Cục Viễn mừng hoặc ấn phím 9 để gặp tư vấn viên để được hỗ trợ của chiến thông. Xin thông báo số thuê bao của quý khách sẽ bị khoá, gọi 102607 để nghe lại (Miễn phí). Chi tiết l/h 9090 (200đ/p)':
    'Bạn nhận được lời nhắn từ 0896543210 vào lúc 13:08 30/09/2023 với nội dung: xin thông báo số thuê bao của quý khách sẽ bị khoá trong 2 giờ tới vui lòng đến Cục Viễn mừng hoặc ấn phím 9 để gặp tư vấn viên để được hỗ trợ của chiến thông. Xin thông báo số thuê bao của quý khách sẽ bị khoá, gọi 102607 để nghe lại (Miễn phí). Chi tiết l/h 9090 (200đ/p)',
    
    # === TIN NHẮN CỜ BẠC / NHÀ CÁI ===
    
    # 24. Quảng cáo nhà cái với [PHONE_NUMBER] và [LINK]
    'V7 top 3 nha cai VN, tuyen dai ly voi muc hoa hong len den 50% , tra hoa hong nhieu hinh thuc , lien he zalo Van: [PHONE_NUMBER] Link: [LINK]':
    'V7 top 3 nha cai VN, tuyen dai ly voi muc hoa hong len den 50% , tra hoa hong nhieu hinh thuc , lien he zalo Van: 0935123456 Link: https://v7bet.vip',
    
    # === TIN NHẮN LỪA ĐẢO SHOPEE / THƯƠNG MẠI ĐIỆN TỬ ===
    
    # 25. Hoàn tiền đơn hàng với [PHONE_NUMBER]
    'https://ww.facebook.com/CSKHduongdinhninh035/ Chị nhắn vào Facebook bạn này ạ chị nhắn là tôi muốn hoàn tiền đơn hàng mĩ phẩm HASAKI bị lỗi gây kích ứng MVD: 1041 SĐT ĐẶT HÀNG: [PHONE_NUMBER]':
    'https://ww.facebook.com/CSKHduongdinhninh035/ Chị nhắn vào Facebook bạn này ạ chị nhắn là tôi muốn hoàn tiền đơn hàng mĩ phẩm HASAKI bị lỗi gây kích ứng MVD: 1041 SĐT ĐẶT HÀNG: 0987654321',
    
    # 26. Trúng thưởng với số điện thoại
    'Xin chuc mung TB[PHONE_NUMBER] Da trung thuong "1 IP14s" trong chuong trinh du thuong cuoi nam Lien He: 1>9>0>0>1>9>9>5 de nhan giai. De nghi LH truoc ngay 28/01':
    'Xin chuc mung TB0912345678 Da trung thuong "1 IP14s" trong chuong trinh du thuong cuoi nam Lien He: 1>9>0>0>1>9>9>5 de nhan giai. De nghi LH truoc ngay 28/01',
    
    # === TIN NHẮN TRÚNG THƯỞNG / KHUYẾN MÃI GIẢ ===
    
    # 27. Trúng thưởng Thế giới di động
    'Chúc mừng anh [NAME] đã trở thành khách hàng may mắn tham gia chương trình vé cào may mắn của thế giới di động https://bit.ly/vecaomayman':
    'Chúc mừng anh Hoàng Minh Tuấn đã trở thành khách hàng may mắn tham gia chương trình vé cào may mắn của thế giới di động https://bit.ly/vecaomayman',
    
    # 28. Hoàn thuế TNCN với [LINK]
    'egeR #Ban da-du D1EU K1EN HOAN~THUE TNCN, tai:[LINK]':
    'egeR #Ban da-du D1EU K1EN HOAN~THUE TNCN, tai:https://hoanthue-tncn.vip',
    
    # 29. Đơn hàng với [PRIVATE_INFO]
    'Đơn hàng số [PRIVATE_INFO] của bạn đã đến kho nhưng không thể giao do địa chỉ không đầy đủ. Cập nhật https://vietteltelecomshop.cfd/':
    'Đơn hàng số SPX2024112567890 của bạn đã đến kho nhưng không thể giao do địa chỉ không đầy đủ. Cập nhật https://vietteltelecomshop.cfd/',
    
    # 30. Shopee trúng thưởng
    'Chúc mừng chị, tài khoản Shopee [NAME]. Shopee xin trân trọng thông báo tài khoản Shopee của quý khách đã rất may mắn nhận được thẻ quà tặng tiền mặt trị giá 200.000.000₫ từ Quà Tặng Shopee 2022. Nhận Quà Tặng Shopee - Truy cập ngay (https://shopee.vn-trangchu.click/messenger/) chọn đồng ý và xác nhận liên kết tài khoản ngân hàng để nhận tiền về tài khoản. Shopee kính gửi!':
    'Chúc mừng chị, tài khoản Shopee nguyenthuha92. Shopee xin trân trọng thông báo tài khoản Shopee của quý khách đã rất may mắn nhận được thẻ quà tặng tiền mặt trị giá 200.000.000₫ từ Quà Tặng Shopee 2022. Nhận Quà Tặng Shopee - Truy cập ngay (https://shopee.vn-trangchu.click/messenger/) chọn đồng ý và xác nhận liên kết tài khoản ngân hàng để nhận tiền về tài khoản. Shopee kính gửi!',
    
    # 31. Số điện thoại liên kết bị thay đổi
    'So dien thoai lien ket voi tai khoan ngan hang cua ban da bi thay doi vao so [PHONE_NUMBER] neu khong phai do ban thay doi vui long nhan vao https://svip- scb.com de huy':
    'So dien thoai lien ket voi tai khoan ngan hang cua ban da bi thay doi vao so 0976543210 neu khong phai do ban thay doi vui long nhan vao https://svip- scb.com de huy',
}

print(f"Tổng số mẫu cần thay thế thủ công: {len(manual_replacements)}")


Tổng số mẫu cần thay thế thủ công: 31


In [26]:
# Áp dụng thay thế thủ công
def apply_manual_replacements(content, replacements_dict):
    """
    Áp dụng thay thế thủ công từ dictionary.
    Nếu content nằm trong dictionary thì trả về giá trị đã thay thế,
    ngược lại trả về content gốc.
    """
    return replacements_dict.get(content, content)

# Tạo cột mới với nội dung đã được thay thế
df['content_augmented'] = df['content'].apply(
    lambda x: apply_manual_replacements(x, manual_replacements)
)

# Kiểm tra số lượng đã được thay thế
num_replaced = (df['content'] != df['content_augmented']).sum()
print(f"Số lượng tin nhắn đã được thay thế: {num_replaced}")
print(f"Số lượng tin nhắn không thay đổi: {len(df) - num_replaced}")


Số lượng tin nhắn đã được thay thế: 31
Số lượng tin nhắn không thay đổi: 219


In [27]:
# Hiển thị so sánh trước/sau cho các mẫu đã thay thế
replaced_mask = df['content'] != df['content_augmented']
df_replaced = df[replaced_mask][['content', 'content_augmented']].copy()

print(f"=== SO SÁNH TRƯỚC/SAU THAY THẾ ({len(df_replaced)} mẫu) ===\n")

for idx, row in df_replaced.iterrows():
    print(f"--- Mẫu {idx + 1} ---")
    print(f"GỐC: {row['content'][:150]}...")
    print(f"SAU: {row['content_augmented'][:150]}...")
    print()

=== SO SÁNH TRƯỚC/SAU THAY THẾ (31 mẫu) ===

--- Mẫu 1 ---
GỐC: [TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông/Bà [NAME] Trước 17h ngày hôm nay không thanh toán dứt điểm số tiền quá hạn 910.046 VNĐ. Chúng tôi đơn p...
SAU: [TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông/Bà Nguyễn Văn Minh Trước 17h ngày hôm nay không thanh toán dứt điểm số tiền quá hạn 910.046 VNĐ. Chúng ...

--- Mẫu 2 ---
GỐC: [TB] Tin ich Loi nham thoai cua Viettel: Quy khach co loi nhan tu TB 0848836182 vao luc 08:09 27/03/2025. Voi noi dung: kính chào quý khách đây là cuộ...
SAU: [TB] Tin ich Loi nham thoai cua Viettel: Quy khach co loi nhan tu TB 0848836182 vao luc 08:09 27/03/2025. Voi noi dung: kính chào quý khách đây là cuộ...

--- Mẫu 3 ---
GỐC: Western Union TB: Vietcombank: [BANK_ACCOUNT]. [NAME]. Ref +19.56 USD. Nhận 500.000 VND. Ngay 02/02/2025. Mgd: 1057425286. Nd: COC TIEN HANG. Quý khác...
SAU: Western Union TB: Vietcombank: 0071000986547. Trần Thị Lan. Ref +19.56 USD. Nhận 500.000 VND. Ngay 02/02/2025. 

In [28]:
# Kiểm tra xem còn placeholder nào chưa được thay thế không
placeholder_keywords = [
    '[NAME]', '[CCCD]', '[LOCATION]', '[PHONE_NUMBER]', 
    '[LINK]', '[MONEY_AMOUNT]', '[BANK_ACCOUNT]', '[DATETIME]', 
    '[PRIVATE_INFO]'
]

pattern = '|'.join(map(re.escape, placeholder_keywords))
remaining_placeholders = df['content_augmented'].str.contains(pattern, regex=True, na=False)

if remaining_placeholders.sum() > 0:
    print(f"⚠️ CÒN {remaining_placeholders.sum()} MẪU CHỨA PLACEHOLDER CHƯA ĐƯỢC THAY THẾ:")
    print("-" * 80)
    for idx, row in df[remaining_placeholders].iterrows():
        print(f"Index {idx}: {row['content_augmented'][:200]}...")
        print()
else:
    print("✅ Tất cả các placeholder đã được thay thế thành công!")

✅ Tất cả các placeholder đã được thay thế thành công!


### Hoàn tất và Lưu dữ liệu
- Thay thế cột `content` bằng `content_augmented` 
- Xóa cột tạm `content_augmented`
- Lưu file CSV đã được xử lý


In [29]:
# Hoàn tất dữ liệu
# Thay thế cột content bằng content_augmented
df['content'] = df['content_augmented']

# Xóa cột tạm
df = df.drop(columns=['content_augmented'])

# Kiểm tra dữ liệu cuối cùng
print("=== THỐNG KÊ DỮ LIỆU SAU KHI XỬ LÝ ===")
print(f"Tổng số mẫu: {len(df)}")
print(f"\nPhân bố nhãn:")
print(df['label'].value_counts())
print(f"\nCác cột trong DataFrame: {df.columns.tolist()}")
print(f"\n5 mẫu đầu tiên:")
df.head()

=== THỐNG KÊ DỮ LIỆU SAU KHI XỬ LÝ ===
Tổng số mẫu: 250

Phân bố nhãn:
label
1.0    227
0.0     23
Name: count, dtype: int64

Các cột trong DataFrame: ['content', 'label', 'has_url', 'has_phone_number', 'sender_type']

5 mẫu đầu tiên:


,content,label,has_url,has_phone_number,sender_type
0,[TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông...,1.0,0,0,personal_number
1,[TB] Tin ich Loi nham thoai cua Viettel: Quy k...,1.0,0,1,shortcode
2,Western Union TB: Vietcombank: 0071000986547. ...,1.0,1,0,personal_number
3,"Bắc, tài khoản tài chính của bạn đã được thêm ...",1.0,1,0,personal_number
4,"[CANH CAO LAN CUOI]:Chung toi da nhac nho, can...",1.0,0,0,personal_number


In [30]:
# Lưu dữ liệu đã xử lý
output_path = './GgSheetData_preprocessed.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ Đã lưu dữ liệu đã xử lý vào: {output_path}")
print(f"   Tổng số dòng: {len(df)}")

# Kiểm tra file đã lưu
import os
file_size = os.path.getsize(output_path) / 1024  # KB
print(f"   Kích thước file: {file_size:.2f} KB")


✅ Đã lưu dữ liệu đã xử lý vào: ./GgSheetData_preprocessed.csv
   Tổng số dòng: 250
   Kích thước file: 60.98 KB
